In [363]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score
from IPython.display import display
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [364]:
df = pd.read_csv("course_lead_scoring.csv")
print(df.info())
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB
None


lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [365]:
df[["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score"]].fillna(0.0, inplace=True)
df.annual_income.fillna(0.0, inplace=True)
df.lead_source.fillna("NA", inplace=True)
df.industry.fillna("NA", inplace=True)
df.employment_status.fillna("NA", inplace=True)
df.location.fillna("NA", inplace=True)

C:\Users\ialnassr\AppData\Local\Temp\ipykernel_24504\2562006015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score"]].fillna(0.0, inplace=True)
C:\Users\ialnassr\AppData\Local\Temp\ipykernel_24504\2562006015.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df

In [366]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   object 
 1   industry                  1462 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1462 non-null   float64
 4   employment_status         1462 non-null   object 
 5   location                  1462 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [367]:
df.columns


Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [368]:
categorical = ['lead_source','industry', 'employment_status', 'location']
df[categorical].head()

,lead_source,industry,employment_status,location
0,paid_ads,NA,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NA,australia
4,referral,education,self_employed,europe


In [369]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   object 
 1   industry                  1462 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1462 non-null   float64
 4   employment_status         1462 non-null   object 
 5   location                  1462 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [370]:
numerical = ["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score"]

Question 1

In [371]:
df[categorical].industry.value_counts(dropna=False)
# retail = 203

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [372]:
courses = df[numerical].number_of_courses_viewed
income = df[numerical].annual_income
interaction = df[numerical].interaction_count
lead_source = df[numerical].lead_score

Question 2

In [373]:

print(df[numerical].interaction_count.corr(df[numerical].lead_score))
print(df[numerical].number_of_courses_viewed.corr(df[numerical].lead_score))
print(df[numerical].number_of_courses_viewed.corr(df[numerical].interaction_count))
print(df[numerical].annual_income.corr(df[numerical].interaction_count))
# 0.009888182496913084
# -0.004878998354681256
# -0.023565222882888044
# 0.027036472404814348
print("annual_income", "interation_count")

0.009888182496913084
-0.004878998354681256
-0.023565222882888044
0.027036472404814348
annual_income interation_count


In [374]:
# first creates a split of 20/80 full/test sets
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
# applying it a second time gives 60/20/20 split
# here we have to use 25% of the original 80% full_train
# to get the desired split. 20/80 = 25
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [375]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']
del df_full_train['converted']

In [376]:
df[categorical].columns

Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object')

Question 3

In [377]:
for x in df_train[categorical]:
    print(x,  ":", mutual_info_score(df_train[categorical][x], y_train))

# lead_source : .035

# adjusted_mutual_info_score(df_full_train.churn, df_full_train.contract)

lead_source : 0.03539624379726594
industry : 0.011574521435657112
employment_status : 0.012937677269442782
location : 0.004464157884038034


In [378]:
# define function, so only one variable is called for function .apply
def mutual_info_churn_score(series):
    return mutual_info_score(series, y_train)


In [379]:
mi = df_train[categorical].apply(mutual_info_churn_score) 
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

Question 4

In [380]:
categorical


['lead_source', 'industry', 'employment_status', 'location']

In [381]:
dv = DictVectorizer(sparse=False) # false to comporessed matrix

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict) 
# equivalent to .fit & .transform

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
#also on validation data

In [382]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [383]:
def logistic_regression(xi):
    score = w0
    
    for j in range(len(w)):
        score = score + xi[j] * w[j]
        
    result = sigmoid(score)
    return result

In [384]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# solver='lbfgs' is the default solver in newer version of sklearn
# for older versions, you need to specify it explicitly
model.fit(X_train, y_train)
model.intercept_[0]
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

Question 5

In [385]:
import itertools

categorical_lists = list(itertools.combinations(categorical, 3))
categorical_industry = categorical_lists[0]
categorical_location = categorical_lists[1]
categorical_employment = categorical_lists[2]
categorical_source = categorical_lists[3]
categorical_location = list(categorical_location)
categorical_employment = list(categorical_employment)
categorical_industry = list(categorical_industry)
categorical_source = list(categorical_source)
categorical_lists = list(categorical_lists)
categorical_lists

[('lead_source', 'industry', 'employment_status'),
 ('lead_source', 'industry', 'location'),
 ('lead_source', 'employment_status', 'location'),
 ('industry', 'employment_status', 'location')]

In [386]:
for x in categorical_lists:
    
    dv = DictVectorizer(sparse=False) # false to comporessed matrix

    train_dict = df_train[list(x) + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict) 
    # equivalent to .fit & .transform

    val_dict = df_val[list(x) + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    #also on validation data

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    # solver='lbfgs' is the default solver in newer version of sklearn
    # for older versions, you need to specify it explicitly
    model.fit(X_train, y_train)
    model.intercept_[0]
    model.coef_[0].round(3)

    from sklearn.metrics import accuracy_score
    model.predict_proba(X_train)
    y_pred = model.predict_proba(X_val)[:, 1]

    converted = (y_pred >= 0.50).astype(int)
    print(x, ":", 0.6996587030716723 - (y_val == converted).mean())


('lead_source', 'industry', 'employment_status') : -0.010238907849829393
('lead_source', 'industry', 'location') : 0.0034129692832763903
('lead_source', 'employment_status', 'location') : 0.0
('industry', 'employment_status', 'location') : -0.0034129692832765013


industry is smallest 0.0

Question 6

In [358]:
for c in [0.01, 0.1, 1, 10, 100]:
    
    dv = DictVectorizer(sparse=False) # false to comporessed matrix

    train_dict = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict) 
    # equivalent to .fit & .transform

    val_dict = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    #also on validation data
    
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    # solver='lbfgs' is the default solver in newer version of sklearn
    # for older versions, you need to specify it explicitly
    model.fit(X_train, y_train)
    model.intercept_[0]
    model.coef_[0]

    from sklearn.metrics import accuracy_score
    model.predict_proba(X_train)
    y_pred = model.predict_proba(X_val)[:, 1]

    converted = (y_pred >= 0.62).astype(int)
    print(f"{c}: {(y_val == converted).mean().round(3)}")

0.01: 0.511
0.1: 0.511
1: 0.512
10: 0.512
100: 0.512


In [355]:
y_val = np.asarray(y_val)

In [357]:
C = [0.01, 0.1, 1, 10, 100]
for c in C:
    dicts_train = df_train[categorical +numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts_train)
    model = LogisticRegression(solver='lbfgs', C=c, max_iter=10000, random_state=42)
    model.fit(X_train, y_train)

    dicts_val = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(dicts_val)
    y_pred = model.predict(X_val)

    acc = (y_val == y_pred).mean()
    print(c, acc)

0.01 0.5167212198161889
0.1 0.5124928653799112
1 0.5101864902328507
10 0.5105708860906941
100 0.5105708860906941


In [342]:
y_val = pd.DataFrame(y_val)
y_val.value_counts(normalize=True)


0
1    0.556314
0    0.443686
Name: proportion, dtype: float64

In [345]:
import numpy as np
from sklearn.metrics import accuracy_score

best_thr = 0
best_acc = 0

for thr in np.arange(0.0, 1.01, 0.01):
    preds = (y_pred_prob >= thr).astype(int)
    acc = accuracy_score(y_val, preds)
    if acc > best_acc:
        best_acc = acc
        best_thr = thr

print(f"Best threshold: {best_thr:.2f}, Best accuracy: {best_acc:.3f}")


Best threshold: 0.62, Best accuracy: 0.795


In [344]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_val, y_pred)
print("ROC AUC:", auc)


ROC AUC: 0.8546956111373288


In [343]:
y_pred_prob = model.predict_proba(X_val)[:, 1]
print(y_pred_prob.min(), y_pred_prob.max())


0.20152555073494396 0.9773244633578183
